# III. Binary File Formats

## 1. pickle
The `pickle` module implements binary protocols for serializing and de-serializing a Python object structure. Only Python can properly read and write pickle files

In [1]:
# Let's create a data frame first
import numpy as np
import pandas as pd

values = np.array([
    [100, 80, 95, 'A'],
    [55, 60, 45, 'F'],
    [70, 75, 90, 'A'],
    [75, 70, 60, 'D'],
    [60, 73, 75, 'C'],
    [72, 63, -1, 'NA']
])
df = pd.DataFrame(values,
                   columns=['Midterm', 'Project', 'Final', 'LetterGrade'],
                   index=['Alex', 'Bob', 'Chris', 'Doug', 'Eva', "Frank"])
df

,Midterm,Project,Final,LetterGrade
Alex,100,80,95,A
Bob,55,60,45,F
Chris,70,75,90,A
Doug,75,70,60,D
Eva,60,73,75,C
Frank,72,63,-1,NA


In [7]:
# Save as a .pickle file
df.to_pickle("data.pickle")

In [8]:
# Load the pickle file
df_pickle = pd.read_pickle("data.pickle")
df_pickle

,Col1,Col2
0,-0.450955,-1.046872
1,-0.389616,-0.329320
2,-0.513321,0.633152
3,1.458837,-1.792533
4,1.116445,-0.015322
...,...,...
95,2.010952,1.673290
96,0.312826,0.418869
97,-0.927486,-2.385562
98,0.999109,-0.063685


## 2. HDF5
The "HDF" stands for "hierarchical data format". HDF5 can be a good choice for working with very large datasets that don't fit into memory, as you can efficiently read and write small sections of large arrays.

In [4]:
df = pd.DataFrame({
    'Col1': np.random.randn(100),
    'Col2': np.random.randn(100)
})
df.head(5)

,Col1,Col2
0,-0.450955,-1.046872
1,-0.389616,-0.329320
2,-0.513321,0.633152
3,1.458837,-1.792533
4,1.116445,-0.015322


In [5]:
# The PyTable package may require update
!pip3 install --upgrade tables

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 11.1 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 10.6 MB/s eta 0:00:00m eta 0:00:010:01:01
  Attempting uninstall: tables
    Found existing installation: tables 3.6.1
    Uninstalling tables-3.6.1:
      Successfully uninstalled tables-3.6.1

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [9]:
df.to_hdf('data.h5', 'obj1', format='table')

In [10]:
df_hdf5 = pd.read_hdf('data.h5', 'obj1', where=['index < 3'])
df_hdf5

,Col1,Col2
0,-0.450955,-1.046872
1,-0.389616,-0.329320
2,-0.513321,0.633152


## 3. feather
The feather format is adapted from the R statistical language. It has extremely high read and write performance.

In [11]:
!pip install -U pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 10.1 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [13]:
import time
start = time.time()
df.to_feather('data.feather')
# df.to_pickle('Data/temp/data2.pickle')
end = time.time()
print("Time cost:", (end - start))

Time cost: 0.0027914047241210938


In [16]:
import time
start = time.time()
df_feather = pd.read_feather('data.feather')
end = time.time()
print("Time cost:", (end - start))
# df_feather

Time cost: 0.0021181106567382812


# III. Interacting with Databases
In a business setting, most data may not be stored in text or binary files. SQL-based relational databases (such as mySQL) are in wide use.

Python has sqlite3 package to interact with databases, and Pandas has some functions to simplify the process.

In [27]:
# Create a SQLite database
import sqlite3
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""
con = sqlite3.connect('data.sqlite')
con.execute(query)
con.commit()

In [28]:
# query = """
# DROP TABLE test
# """
# con.execute(query)
# con.commit()

In [29]:
# Insert a few rows of data
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [30]:
# Select data
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [31]:
# Retrieve columns names
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [32]:
# Extract column names from description

# Solution 1: use a loop
desc = cursor.description
cols = []
for elt in desc:
#     print(elt[0])
    cols.append(elt[0])
print(cols)

['a', 'b', 'c', 'd']


In [33]:
cols = [elt[0] for elt in cursor.description]
print(cols)

['a', 'b', 'c', 'd']


In [34]:
# Create a pandas data frame
columns = [x[0] for x in cursor.description]
df = pd.DataFrame(rows, columns=columns)
df

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


Use package `sqlalchemy` to create a data frame directly from a database.

In [35]:
!pip install sqlalchemy


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [36]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///data.sqlite')
df = pd.read_sql('select * from test', db)
df

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
